# HW 6

## Imports

In [1]:
import nbtools
nbtools.setup_nb()

In [2]:
import itertools
from astropy import units
import sympy
import pandas
from scipy import integrate
import numpy
import plotly.express as px
from plotly import graph_objects as go

from sympy.diffgeom import Manifold, Patch
from pystein import coords, metric, curvature, geodesic
from pystein.utilities import tensor_pow as tpow, full_simplify, boundary_filter

## Exercises B

### B3

In [3]:
M = Manifold('M', dim=4)
P = Patch('origin', M)

t, r, theta, phi, k = sympy.symbols('t r theta phi k')
cs = coords.CoordSystem('spherical', P, [t, r, theta, phi])
dt, dr, dtheta, dphi = cs.base_oneforms()
a = sympy.Function('a')(t)


ds2 = - tpow(dt, 2) + a ** 2 * ((1 / (1 - k * r ** 2)) * tpow(dr, 2) + 
                                r**2 * tpow(dtheta, 2) + 
                                r ** 2 * sympy.sin(theta) ** 2 * tpow(dphi, 2))
g = metric.Metric(twoform=ds2)
g

a(t)**2*(r**2*sin(theta)**2*TensorProduct(dphi, dphi) + r**2*TensorProduct(dtheta, dtheta) + TensorProduct(dr, dr)/(-k*r**2 + 1)) - TensorProduct(dt, dt)

In [4]:
crs = curvature.compute_components(g)

KeyboardInterrupt: 

In [11]:
christoffels = []
for m, n, l in itertools.product(range(4), range(4), range(4)):
    print(m, n, l)
    gamma = sympy.simplify(curvature.christoffel_symbol_component(l, m, n, metric=g).doit())
    christoffels.append(((m, n, l), gamma))
christoffels = [x for x in christoffels if x[1]]

0 0 0
0 0 1
0 0 2
0 0 3
0 1 0
0 1 1
0 1 2
0 1 3
0 2 0
0 2 1
0 2 2
0 2 3
0 3 0
0 3 1
0 3 2
0 3 3
1 0 0
1 0 1
1 0 2
1 0 3
1 1 0
1 1 1
1 1 2
1 1 3
1 2 0
1 2 1
1 2 2
1 2 3
1 3 0
1 3 1
1 3 2
1 3 3
2 0 0
2 0 1
2 0 2
2 0 3
2 1 0
2 1 1
2 1 2
2 1 3
2 2 0
2 2 1
2 2 2
2 2 3
2 3 0
2 3 1
2 3 2
2 3 3
3 0 0
3 0 1
3 0 2
3 0 3
3 1 0
3 1 1
3 1 2
3 1 3
3 2 0
3 2 1
3 2 2
3 2 3
3 3 0
3 3 1
3 3 2
3 3 3


In [12]:
curvature.display_components(christoffels)

In [21]:
def covariant_deriv(mu, nu, vector, metric):
    symbols = metric.coord_system.base_symbols()
    
    x_mu = symbols[mu]
    v_nu = vector[nu]
    
    res = sympy.Derivative(v_nu, x_mu)
    
    for alpha in range(4):
        v_alpha = vector[alpha]
        gamma_mu_alpha_nu = sympy.simplify(curvature.christoffel_symbol_component(nu, mu, alpha, metric=g).doit())
        res += gamma_mu_alpha_nu * v_alpha

    res = sympy.simplify(res.doit())
        
    return res

In [16]:
eta_vector = [
    0, 
    sympy.sqrt(1 - k * r**2) * sympy.cos(theta), 
    - sympy.sqrt(1 - k * r**2) * sympy.sin(theta) / r, 
    0,
]

In [27]:
def cov_deriv_matrix(vector, g):
    components = [4 * [None] for i in range(4)]
    for mu in range(4):
        for nu in range(4):
            del_mu_v_nu = covariant_deriv(mu, nu, vector, g)
            components[mu][nu] = del_mu_v_nu
    return sympy.Matrix(components)

In [29]:
cmat = cov_deriv_matrix(eta_vector, g)

In [31]:
cmat * g.matrix

Matrix([
[                                                      0, a(t)*cos(theta)*Derivative(a(t), t)/sqrt(-k*r**2 + 1), -r*sqrt(-k*r**2 + 1)*a(t)*sin(theta)*Derivative(a(t), t), 0],
[ -a(t)*cos(theta)*Derivative(a(t), t)/sqrt(-k*r**2 + 1),                                                     0,              k*r**2*a(t)**2*sin(theta)/sqrt(-k*r**2 + 1), 0],
[r*sqrt(-k*r**2 + 1)*a(t)*sin(theta)*Derivative(a(t), t),          -k*r**2*a(t)**2*sin(theta)/sqrt(-k*r**2 + 1),                                                        0, 0],
[                                                      0,                                                     0,                                                        0, 0]])